# Análisis del Grafo de Prendas de Vestir

Este notebook presenta un análisis del grafo de prendas utilizando diversos algoritmos de NetworkX. El grafo representa prendas como nodos y sus compatibilidades como aristas ponderadas (peso de 0 a 1, donde mayor peso indica combinación más arriesgada/creativa).

**Estructura del grafo:**
- **Nodos**: Prendas con atributos (talla, color, tipo, estilo, estampado)
- **Aristas**: Compatibilidad entre prendas (ponderadas 0-1)
- **Triángulos**: Conjuntos de 3 prendas que combinan bien (looks)

**Niveles de prendas:**
- Nivel 1: Parte inferior y prendas completas (pantalones, faldas, vestidos, monos)
- Nivel 2: Parte superior (camisetas, jerseys, blusas, camisas)
- Nivel 3: Complementos (chaquetas, abrigos, bufandas)

In [3]:
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import combinations
import warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

## 1. Carga del Grafo

Cargamos el grafo desde el archivo .gexf que contiene toda la información de prendas y sus relaciones.

In [4]:
# Cargar el grafo desde archivo GEXF
# IMPORTANTE: Ajusta la ruta según donde tengas tu archivo
G = nx.read_gexf('grafo_prendas.gexf')

# Información básica del grafo
print("="*60)
print("INFORMACIÓN BÁSICA DEL GRAFO")
print("="*60)
print(f"Número de nodos (prendas): {G.number_of_nodes()}")
print(f"Número de aristas (combinaciones): {G.number_of_edges()}")
print(f"Tipo de grafo: {'Dirigido' if G.is_directed() else 'No dirigido'}")
print(f"Densidad del grafo: {nx.density(G):.4f}")
print(f"\nNúmero de componentes conexas: {nx.number_connected_components(G)}")

INFORMACIÓN BÁSICA DEL GRAFO
Número de nodos (prendas): 7555
Número de aristas (combinaciones): 1140880
Tipo de grafo: No dirigido
Densidad del grafo: 0.0400

Número de componentes conexas: 1


## 2. Estadísticas Descriptivas del Grafo

Analizamos las propiedades estructurales básicas del grafo para entender su topología.

In [ ]:
# Distribución de grados
degrees = dict(G.degree())
degree_values = list(degrees.values())

print("="*60)
print("DISTRIBUCIÓN DE GRADOS")
print("="*60)
print(f"Grado promedio: {np.mean(degree_values):.2f}")
print(f"Grado mediano: {np.median(degree_values):.2f}")
print(f"Desviación estándar: {np.std(degree_values):.2f}")
print(f"Grado mínimo: {min(degree_values)}")
print(f"Grado máximo: {max(degree_values)}")

# Top 10 prendas más versátiles (mayor grado)
print("\n" + "="*60)
print("TOP 10 PRENDAS MÁS VERSÁTILES (más combinaciones)")
print("="*60)
top_degree = sorted(degrees.items(), key=lambda x: x[1], reverse=True)[:10]
for i, (node, degree) in enumerate(top_degree, 1):
    print(f"{i:2d}. Prenda {node}: {degree} combinaciones")

DISTRIBUCIÓN DE GRADOS
Grado promedio: 302.02
Grado mediano: 182.00
Desviación estándar: 391.93
Grado mínimo: 8
Grado máximo: 4291

TOP 10 PRENDAS MÁS VERSÁTILES (más combinaciones)
 1. Prenda 3408: 4291 combinaciones
 2. Prenda 3773: 4291 combinaciones
 3. Prenda 1807: 3749 combinaciones
 4. Prenda 1816: 3749 combinaciones
 5. Prenda 1846: 3749 combinaciones
 6. Prenda 1857: 3749 combinaciones
 7. Prenda 6386: 3749 combinaciones
 8. Prenda 1826: 3604 combinaciones
 9. Prenda 1862: 3604 combinaciones
10. Prenda 1868: 3604 combinaciones


In [ ]:
# Análisis de pesos de las aristas
weights = [G[u][v]['weight'] for u, v in G.edges()]

print("="*60)
print("ANÁLISIS DE PESOS DE COMBINACIONES")
print("="*60)
print(f"Peso promedio: {np.mean(weights):.4f}")
print(f"Peso mediano: {np.median(weights):.4f}")
print(f"Desviación estándar: {np.std(weights):.4f}")
print(f"Peso mínimo: {min(weights):.4f}")
print(f"Peso máximo: {max(weights):.4f}")

ANÁLISIS DE PESOS DE COMBINACIONES
Peso promedio: 0.5454
Peso mediano: 0.5200
Desviación estándar: 0.1770
Peso mínimo: 0.1400
Peso máximo: 0.8800

Interpretación: Pesos cercanos a 1 = combinaciones más arriesgadas/creativas
                 Pesos cercanos a 0 = combinaciones más conservadoras


## 3. Análisis de Triángulos (Looks Potenciales)

Los triángulos en nuestro grafo representan conjuntos de 3 prendas que pueden formar un look completo. Analizamos su distribución y características.

**Justificación del método**: El conteo de triángulos es fundamental porque:
- Cada triángulo representa un look válido (3 prendas compatibles entre sí)
- El coeficiente de clustering nos indica qué tan "agrupables" son las prendas
- Las prendas con alto clustering tienden a formar múltiples looks

In [ ]:
# Conteo de triángulos por nodo
triangles_per_node = nx.triangles(G)
total_triangles = sum(triangles_per_node.values()) // 3  # Cada triángulo se cuenta 3 veces

print("="*60)
print("ANÁLISIS DE TRIÁNGULOS (LOOKS POTENCIALES)")
print("="*60)
print(f"Total de triángulos en el grafo: {total_triangles:,}")
print(f"Promedio de triángulos por prenda: {np.mean(list(triangles_per_node.values())):.2f}")
print(f"\nInterpretación: Cada triángulo representa un look válido de 3 prendas")

# Top prendas que participan en más triángulos (más looks)
print("\n" + "="*60)
print("TOP 10 PRENDAS QUE PARTICIPAN EN MÁS LOOKS")
print("="*60)
top_triangles = sorted(triangles_per_node.items(), key=lambda x: x[1], reverse=True)[:10]
for i, (node, count) in enumerate(top_triangles, 1):
    print(f"{i:2d}. Prenda {node}: {count} looks posibles")


ANÁLISIS DE TRIÁNGULOS (LOOKS POTENCIALES)
Total de triángulos en el grafo: 36,040,456
Promedio de triángulos por prenda: 14311.23

Interpretación: Cada triángulo representa un look válido de 3 prendas

TOP 10 PRENDAS QUE PARTICIPAN EN MÁS LOOKS
 1. Prenda 3408: 630388 looks posibles
 2. Prenda 3773: 630388 looks posibles
 3. Prenda 1807: 363357 looks posibles
 4. Prenda 1816: 363357 looks posibles
 5. Prenda 1846: 363357 looks posibles
 6. Prenda 1857: 363357 looks posibles
 7. Prenda 6386: 363357 looks posibles
 8. Prenda 781: 343084 looks posibles
 9. Prenda 1826: 338365 looks posibles
10. Prenda 1862: 338365 looks posibles


## 4. Coeficiente de Clustering

**Justificación del algoritmo**: El coeficiente de clustering mide la tendencia de las prendas a formar grupos cerrados (triángulos). Es crucial porque:
- Alto clustering → la prenda forma muchos looks completos
- Bajo clustering → la prenda combina con muchas otras pero no forman grupos de 3
- Nos ayuda a identificar prendas "cohesionadoras" en el armario

In [ ]:
# Coeficiente de clustering
clustering_coef = nx.clustering(G)
avg_clustering = nx.average_clustering(G)

print("="*60)
print("COEFICIENTE DE CLUSTERING")
print("="*60)
print(f"Coeficiente de clustering promedio: {avg_clustering:.4f}")
print(f"\nInterpretación:")
print(f"  - Valor alto (>0.5): Las prendas tienden a formar grupos cohesivos")
print(f"  - Valor bajo (<0.3): Las conexiones son más dispersas")

# Top prendas con mayor clustering
print("\n" + "="*60)
print("TOP 10 PRENDAS MÁS COHESIONADORAS (alto clustering)")
print("="*60)
top_clustering = sorted(clustering_coef.items(), key=lambda x: x[1], reverse=True)[:10]
for i, (node, coef) in enumerate(top_clustering, 1):
    print(f"{i:2d}. Prenda {node}: {coef:.4f}")

COEFICIENTE DE CLUSTERING
Coeficiente de clustering promedio: 0.2812

Interpretación:
  - Valor alto (>0.5): Las prendas tienden a formar grupos cohesivos
  - Valor bajo (<0.3): Las conexiones son más dispersas

TOP 10 PRENDAS MÁS COHESIONADORAS (alto clustering)
 1. Prenda 2832: 0.7500
 2. Prenda 3588: 0.7500
 3. Prenda 4411: 0.7500
 4. Prenda 5138: 0.7500
 5. Prenda 1375: 0.6637
 6. Prenda 2099: 0.6599
 7. Prenda 1368: 0.6558
 8. Prenda 6369: 0.6545
 9. Prenda 2752: 0.6467
10. Prenda 2747: 0.6465


## 5. Análisis de Centralidad

Aplicamos múltiples medidas de centralidad para identificar las prendas más "importantes" desde diferentes perspectivas.

**Justificación de los algoritmos**:
- **Centralidad de grado**: Prendas con más combinaciones directas (básicos del armario)
- **Centralidad de intermediación**: Prendas que "conectan" diferentes grupos de estilo
- **Centralidad de cercanía**: Prendas accesibles a todo el armario

In [9]:
# 5.1 Centralidad de Grado
degree_centrality = nx.degree_centrality(G)

print("="*60)
print("CENTRALIDAD DE GRADO")
print("="*60)
print("Identifica los 'básicos del armario' - prendas con más combinaciones directas\n")

top_degree_cent = sorted(degree_centrality.items(), key=lambda x: x[1], reverse=True)[:10]
print("TOP 10 PRENDAS (Centralidad de Grado):")
for i, (node, cent) in enumerate(top_degree_cent, 1):
    print(f"{i:2d}. Prenda {node}: {cent:.4f}")

CENTRALIDAD DE GRADO
Identifica los 'básicos del armario' - prendas con más combinaciones directas

TOP 10 PRENDAS (Centralidad de Grado):
 1. Prenda 3408: 0.5680
 2. Prenda 3773: 0.5680
 3. Prenda 1807: 0.4963
 4. Prenda 1816: 0.4963
 5. Prenda 1846: 0.4963
 6. Prenda 1857: 0.4963
 7. Prenda 6386: 0.4963
 8. Prenda 1826: 0.4771
 9. Prenda 1862: 0.4771
10. Prenda 1868: 0.4771


In [10]:
# 5.2 Centralidad de Intermediación
betweenness_centrality = nx.betweenness_centrality(G, weight='weight')

print("="*60)
print("CENTRALIDAD DE INTERMEDIACIÓN")
print("="*60)
print("Identifica prendas 'puente' que conectan diferentes estilos\n")

top_betweenness = sorted(betweenness_centrality.items(), key=lambda x: x[1], reverse=True)[:10]
print("TOP 10 PRENDAS (Centralidad de Intermediación):")
for i, (node, cent) in enumerate(top_betweenness, 1):
    print(f"{i:2d}. Prenda {node}: {cent:.4f}")

CENTRALIDAD DE INTERMEDIACIÓN
Identifica prendas 'puente' que conectan diferentes estilos

TOP 10 PRENDAS (Centralidad de Intermediación):
 1. Prenda 1455: 0.0686
 2. Prenda 1458: 0.0686
 3. Prenda 1464: 0.0681
 4. Prenda 1465: 0.0681
 5. Prenda 1457: 0.0528
 6. Prenda 1460: 0.0528
 7. Prenda 1807: 0.0491
 8. Prenda 1816: 0.0491
 9. Prenda 1846: 0.0491
10. Prenda 1857: 0.0491


In [11]:
# 5.3 Centralidad de Cercanía
closeness_centrality = nx.closeness_centrality(G, distance='weight')

print("="*60)
print("CENTRALIDAD DE CERCANÍA")
print("="*60)
print("Identifica prendas 'accesibles' que están cerca de todo el armario\n")

top_closeness = sorted(closeness_centrality.items(), key=lambda x: x[1], reverse=True)[:10]
print("TOP 10 PRENDAS (Centralidad de Cercanía):")
for i, (node, cent) in enumerate(top_closeness, 1):
    print(f"{i:2d}. Prenda {node}: {cent:.4f}")

CENTRALIDAD DE CERCANÍA
Identifica prendas 'accesibles' que están cerca de todo el armario

TOP 10 PRENDAS (Centralidad de Cercanía):
 1. Prenda 1807: 2.9253
 2. Prenda 1816: 2.9253
 3. Prenda 1846: 2.9253
 4. Prenda 1857: 2.9253
 5. Prenda 6386: 2.9253
 6. Prenda 1826: 2.9003
 7. Prenda 1862: 2.9003
 8. Prenda 1868: 2.9003
 9. Prenda 1869: 2.9003
10. Prenda 6406: 2.9003


## 7 PageRank

PageRank identifica las prendas más "influyentes" del armario considerando no solo cuántas conexiones tienen, sino también la importancia de las prendas con las que se conectan. Una prenda con PageRank alto es una pieza clave que conecta con otras prendas importantes.

**Interpretación**: Una prenda con PageRank alto es como un "hub" de estilo que no solo tiene muchas combinaciones, sino que esas combinaciones son con otras prendas también importantes.

In [ ]:
# Calcular PageRank con alpha=0.9 (damping factor)
print("Calculando PageRank...")
pagerank = nx.pagerank(G, alpha=0.9, weight='weight')

print("="*60)
print("CENTRALIDAD POR PAGERANK")
print("="*60)
print("Identifica las prendas más 'influyentes' del armario\n")
print("Una prenda con alto PageRank no solo tiene muchas combinaciones,")
print("sino que se combina con otras prendas también importantes.\n")

# Estadísticas de PageRank
pr_values = list(pagerank.values())
print(f"PageRank promedio: {np.mean(pr_values):.6f}")
print(f"PageRank máximo: {np.max(pr_values):.6f}")
print(f"PageRank mínimo: {np.min(pr_values):.6f}")
print(f"Desviación estándar: {np.std(pr_values):.6f}")

# Top 20 prendas con mayor PageRank
print("\n" + "="*60)
print("TOP 20 PRENDAS CON MAYOR PAGERANK (Más Influyentes)")
print("="*60)
top_pagerank = sorted(pagerank.items(), key=lambda x: x[1], reverse=True)[:20]
for i, (node, pr) in enumerate(top_pagerank, 1):
    # También mostrar el grado para comparar
    node_degree = degrees[node]
    print(f"{i:2d}. Prenda {node}: PR={pr:.6f} (grado: {node_degree})")

# Comparación con otras centralidades para las top 10
print("\n" + "="*60)
print("COMPARACIÓN DE MÉTRICAS - TOP 5 PAGERANK")
print("="*60)
print(f"{'Prenda':<10} {'PageRank':<12} {'Grado':<8} {'Deg.Cent':<10} {'Clustering':<12}")
print("-"*60)
for node, pr in top_pagerank[:5]:
    print(f"{node:<10} {pr:<12.6f} {degrees[node]:<8} {degree_centrality[node]:<10.4f} {clustering_coef[node]:<12.4f}")

print("\n INSIGHT: Las prendas con alto PageRank son los 'conectores clave'")
print("   del armario - son versátiles Y se conectan con otras prendas versátiles.")

Calculando PageRank...
CENTRALIDAD POR PAGERANK
Identifica las prendas más 'influyentes' del armario

Una prenda con alto PageRank no solo tiene muchas combinaciones,
sino que se combina con otras prendas también importantes.

PageRank promedio: 0.000132
PageRank máximo: 0.001711
PageRank mínimo: 0.000017
Desviación estándar: 0.000140

TOP 20 PRENDAS CON MAYOR PAGERANK (Más Influyentes)
 1. Prenda 3408: PR=0.001711 (grado: 4291)
 2. Prenda 3773: PR=0.001711 (grado: 4291)
 3. Prenda 3549: PR=0.001551 (grado: 2234)
 4. Prenda 3551: PR=0.001416 (grado: 2135)
 5. Prenda 3516: PR=0.001367 (grado: 2776)
 6. Prenda 781: PR=0.001356 (grado: 3002)
 7. Prenda 1308: PR=0.001355 (grado: 2041)
 8. Prenda 3796: PR=0.001327 (grado: 2691)
 9. Prenda 495: PR=0.001257 (grado: 2781)
10. Prenda 3467: PR=0.001250 (grado: 2781)
11. Prenda 3793: PR=0.001199 (grado: 2730)
12. Prenda 3794: PR=0.001174 (grado: 2674)
13. Prenda 955: PR=0.001135 (grado: 2565)
14. Prenda 1716: PR=0.001135 (grado: 2565)
15. Prenda 

## 8. Resultados

In [ ]:
# Crear DataFrame con todas las métricas
metrics_df = pd.DataFrame({
    'prenda_id': list(G.nodes()),
    'grado': [degrees[n] for n in G.nodes()],
    'triangulos': [triangles_per_node[n] for n in G.nodes()],
    'clustering': [clustering_coef[n] for n in G.nodes()],
    'degree_centrality': [degree_centrality[n] for n in G.nodes()],
    'betweenness_centrality': [betweenness_centrality[n] for n in G.nodes()],
    'closeness_centrality': [closeness_centrality[n] for n in G.nodes()],
})

# Ordenar por número de triángulos (looks potenciales)
metrics_df = metrics_df.sort_values('triangulos', ascending=False)

print("Primeras 10 prendas por métricas clave:")
print(metrics_df.head(10))

# Guardar a CSV
metrics_df.to_csv('metricas_prendas.csv', index=False)
print("\n✅ Métricas guardadas en 'metricas_prendas.csv'")

Primeras 10 prendas por métricas clave:
     prenda_id  grado  triangulos  clustering  degree_centrality  \
3407      3408   4291      630388    0.068489           0.568043   
3772      3773   4291      630388    0.068489           0.568043   
1806      1807   3749      363357    0.051719           0.496293   
1815      1816   3749      363357    0.051719           0.496293   
6385      6386   3749      363357    0.051719           0.496293   
1845      1846   3749      363357    0.051719           0.496293   
1856      1857   3749      363357    0.051719           0.496293   
780        781   3002      343084    0.076165           0.397405   
1867      1868   3604      338365    0.052115           0.477098   
1825      1826   3604      338365    0.052115           0.477098   

      betweenness_centrality  closeness_centrality  comunidad  
3407                0.008388              2.357484          0  
3772                0.008388              2.357484          0  
1806               